<h1 align=center> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Project Topic: Stock Pricing using CAPM in China Stock Market</h1>
<h3 align=center> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Author: JeffLin99, Email: jeffhengyu1999@hotmail.com</h3>

## Project Description:
### In this project, the goal is addressed as below:

<p>(1) A selection of factors is made, then a decile portfolio is constructed based on the factor sizes, eventually tests are made to see if there is any significant difference in the portfolio returns.
<p>(2) Application of spanning test, calculating factor alpha
<p>(3) Application of Fama-Macbeth regression test

<h3>Financial Indicator Factor

In [1]:
import pandas as pd
import numpy as np

finidx = pd.read_csv('Data\CSR_Finidx.csv', squeeze = True)
finidx = finidx.rename(columns={'Accper':'Trdmnt'})
finidx = finidx.dropna(axis = 0)
finidx.set_index('Trdmnt', inplace = True)
finidx.index = pd.to_datetime(finidx.index)

#Conversion of annual financial indicator data into monthly data through linear interpolation methods
finidx = finidx.groupby('Stkcd').resample('M').interpolate()
del finidx['Stkcd']
finidx = finidx.reset_index()
finidx['Trdmnt'] = finidx['Trdmnt'].astype(str)
finidx['Trdmnt'] = [i[:7] for i in finidx['Trdmnt']]
finidx

C:\Users\Myth.God\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Myth.God\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\Myth.God\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


,Stkcd,Trdmnt,A100000,A200000,A210000,A210101,A212101,A220000,A220101,A220201,...,T40803,T60200,T40902,Surplus,Flua10,Intrsta10,Perdebt,Begdebt,Enddebt,Outcap
0,2,2002-12,8.215822e+09,4.789363e+09,3.055543e+09,4.600000e+08,0.000000e+00,1.672342e+09,160000000.0,1.512342e+09,...,0.117580,0.630847,1.603991,5.144866e+08,0.941975,-2.904666e+08,1.873171e+09,1.614000e+09,2.132342e+09,5.180719e+07
1,2,2003-01,8.411257e+09,4.873631e+09,3.198981e+09,5.616667e+08,1.333333e+07,1.614104e+09,168350000.0,1.445754e+09,...,0.118964,0.612061,1.628730,5.407433e+08,0.944049,-4.411339e+08,1.923150e+09,1.657195e+09,2.189104e+09,5.094811e+07
2,2,2003-02,8.606692e+09,4.957899e+09,3.342419e+09,6.633333e+08,2.666667e+07,1.555867e+09,176700000.0,1.379167e+09,...,0.120348,0.593274,1.653469,5.670000e+08,0.946122,-5.918011e+08,1.973129e+09,1.700390e+09,2.245867e+09,5.008903e+07
3,2,2003-03,8.802127e+09,5.042168e+09,3.485857e+09,7.650000e+08,4.000000e+07,1.497630e+09,185050000.0,1.312580e+09,...,0.121733,0.574488,1.678208,5.932566e+08,0.948196,-7.424684e+08,2.023108e+09,1.743585e+09,2.302630e+09,4.922995e+07
4,2,2003-04,8.997562e+09,5.126436e+09,3.629296e+09,8.666667e+08,5.333333e+07,1.439393e+09,193400000.0,1.245993e+09,...,0.123117,0.555701,1.702946,6.195133e+08,0.950269,-8.931357e+08,2.073087e+09,1.786781e+09,2.359393e+09,4.837087e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333935,900957,2009-08,7.035093e+08,2.212731e+08,2.195397e+08,1.440000e+08,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,-0.023794,-0.030230,-0.146594,-1.435185e+05,0.286383,7.062145e+05,1.406667e+08,1.373333e+08,1.440000e+08,2.321594e+07
333936,900957,2009-09,7.036796e+08,2.224463e+08,2.204963e+08,1.437500e+08,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,-0.031071,-0.036999,-0.112973,-2.399553e+06,0.288540,9.979218e+06,1.416250e+08,1.395000e+08,1.437500e+08,2.602433e+07
333937,900957,2009-10,7.038499e+08,2.236195e+08,2.214529e+08,1.435000e+08,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,-0.038347,-0.043767,-0.079352,-4.655587e+06,0.290697,1.925222e+07,1.425833e+08,1.416667e+08,1.435000e+08,2.883272e+07
333938,900957,2009-11,7.040203e+08,2.247928e+08,2.224094e+08,1.432500e+08,0.000000e+00,0.000000e+00,0.0,0.000000e+00,...,-0.045624,-0.050536,-0.045731,-6.911621e+06,0.292854,2.852522e+07,1.435417e+08,1.438333e+08,1.432500e+08,3.164111e+07


### Selection of factors:
<p> 1. Net cash flows from operating activities (NOCF);
<p> 2. Return on net assets (ROE);
<p> 3, Growth rate of main income (CFGR);
<p> 4, capital expenditure (Outcap)

In [2]:
choice = ['D610000','T40802','T40902','Outcap']
factor = finidx[['Stkcd','Trdmnt']+choice]
factor = factor.rename(columns={'D610000':'NOCF','T40802':'ROE','T40902':'CRGR'})
factor

,Stkcd,Trdmnt,NOCF,ROE,CRGR,Outcap
0,2,2002-12,1.288433e+08,0.117580,1.603991,5.180719e+07
1,2,2003-01,-5.092322e+06,0.118792,1.628730,5.094811e+07
2,2,2003-02,-1.390279e+08,0.120005,1.653469,5.008903e+07
3,2,2003-03,-2.729635e+08,0.121217,1.678208,4.922995e+07
4,2,2003-04,-4.068991e+08,0.122429,1.702946,4.837087e+07
...,...,...,...,...,...,...
333935,900957,2009-08,-6.883392e+06,-0.023794,-0.146594,2.321594e+07
333936,900957,2009-09,-6.970366e+04,-0.031071,-0.112973,2.602433e+07
333937,900957,2009-10,6.743985e+06,-0.038347,-0.079352,2.883272e+07
333938,900957,2009-11,1.355767e+07,-0.045624,-0.045731,3.164111e+07


<h3>Monthly data on specific stocks</h3>

In [3]:
stk_pool = pd.read_csv('Data\TRD_Mnth.csv')
stk_pool['Mretwd'] = stk_pool['Mretwd'].dropna(axis=0)
stk_pool

,Stkcd,Trdmnt,Msmvosd,Mretwd,Markettype,Ahshrtrd_M
0,1,2010-01,63453279.51,-0.109561,4,NaN
1,1,2010-02,65646365.20,0.034562,4,NaN
2,1,2010-03,67839450.90,0.033408,4,NaN
3,1,2010-04,60119789.25,-0.113793,4,NaN
4,1,2010-05,51201240.75,-0.148346,4,NaN
...,...,...,...,...,...,...
477858,900957,2022-05,101936.00,0.069499,2,0.0
477859,900957,2022-06,110032.00,0.079422,2,0.0
477860,900957,2022-07,107456.00,-0.023408,2,0.0
477861,900957,2022-08,117576.00,0.094179,2,0.0


<h3>Merge stock data and factor data by security code and date

In [4]:
import datetime
alldata = pd.merge(stk_pool, factor, on=['Stkcd','Trdmnt'], how='inner')
alldata.set_index('Trdmnt',inplace = True)
alldata.index = pd.to_datetime(alldata.index, format='%Y-%m')
alldata['Trdmnt'] = alldata.index.astype(str)
alldata

,Stkcd,Msmvosd,Mretwd,Markettype,Ahshrtrd_M,NOCF,ROE,CRGR,Outcap,Trdmnt
Trdmnt,,,,,,,,,,
2010-01-01,2,90187926.46,-0.135985,4,NaN,8.668677e+09,0.155938,0.180914,7.607185e+08,2010-01-01
2010-02-01,2,91056975.00,0.009636,4,NaN,8.084002e+09,0.157988,0.167862,7.153749e+08,2010-02-01
2010-03-01,2,91732901.65,0.007423,4,NaN,7.499327e+09,0.160038,0.154810,6.700313e+08,2010-03-01
2010-04-01,2,75317540.30,-0.178947,4,NaN,6.914653e+09,0.162088,0.141758,6.246876e+08,2010-04-01
2010-05-01,2,69620444.30,-0.066826,4,NaN,6.329978e+09,0.164138,0.128706,5.793440e+08,2010-05-01
...,...,...,...,...,...,...,...,...,...,...
2015-08-01,900956,125810.00,-0.215769,2,NaN,4.029782e+08,0.069690,-0.075302,4.178922e+08,2015-08-01
2015-09-01,900956,125005.00,-0.006399,2,NaN,4.276712e+08,0.070276,-0.064907,4.383700e+08,2015-09-01
2015-10-01,900956,166520.00,0.332110,2,NaN,4.523641e+08,0.070862,-0.054512,4.588478e+08,2015-10-01


### Defining the regression model

In [5]:
def doreg(data):
    mod = sm.OLS(data['Mretwd'], sm.add_constant(data[['NOCF','ROE','CRGR','Outcap']]))
    res = mod.fit()
    return res.params

#### Set training start date, test start date, and test end date.

In [6]:
train_start = '2010-01'
test_start = '2020-01'
test_end = '2021-12'

all_period = alldata.loc[train_start:test_end].index.unique()
train_len = len(alldata.loc[train_start:test_start].index.unique())
test_len = len(alldata.loc[test_start:test_end].index.unique())

varx = ['NOCF','ROE','CRGR','Outcap']

## （In-Sample）Rolling estimate of gamma(Roll) (same period) using 10 years of historical data (same period)

#### Begin rolling regression 

In [7]:
import statsmodels.api as sm

allregres = pd.DataFrame()

for i in range(test_len):
    train_period = all_period[i:train_len+i]
    traindata = alldata[(alldata.index>=train_period[0]) & (alldata.index<=train_period[-1])]

    ##Some filters can be added here
    traindata['nobs'] = traindata.groupby('Stkcd')['Trdmnt'].transform(len)
    traindata=traindata[traindata['nobs']>=30]

    regres = traindata.groupby('Stkcd').apply(doreg)
    for j in varx:
        regres.rename({j:'gamma_'+j},inplace=True,axis=1)
    if len(traindata)>0:
        regres['Trdmnt'] = str(train_period[-1])[:10]

    allregres = pd.concat((allregres,regres), axis=0)
    allregres.dropna(axis=0, inplace=True)

<ipython-input-7-d9639ff36977>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traindata['nobs'] = traindata.groupby('Stkcd')['Trdmnt'].transform(len)
<ipython-input-7-d9639ff36977>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traindata['nobs'] = traindata.groupby('Stkcd')['Trdmnt'].transform(len)
<ipython-input-7-d9639ff36977>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

<ipython-input-7-d9639ff36977>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traindata['nobs'] = traindata.groupby('Stkcd')['Trdmnt'].transform(len)
<ipython-input-7-d9639ff36977>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traindata['nobs'] = traindata.groupby('Stkcd')['Trdmnt'].transform(len)
<ipython-input-7-d9639ff36977>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [8]:
allregres

,const,gamma_NOCF,gamma_ROE,gamma_CRGR,gamma_Outcap,Trdmnt
Stkcd,,,,,,
2,0.072719,1.129567e-12,-0.523720,0.138200,-4.313307e-12,2020-07-01
4,0.135703,2.915632e-09,1.454471,-0.022265,-1.170533e-08,2020-07-01
5,-0.068700,4.510647e-10,-0.651972,0.068448,1.502041e-09,2020-07-01
6,-0.001943,-2.123810e-11,0.162049,0.002011,-2.028509e-10,2020-07-01
7,0.092080,-6.995901e-11,-0.583141,0.064471,-5.465540e-08,2020-07-01
...,...,...,...,...,...,...
900951,0.217498,-1.484403e-09,-0.155983,-0.025485,-1.026258e-09,2016-06-01
900952,0.210518,-3.399996e-10,-1.163805,-0.003496,9.087156e-13,2016-06-01
900953,0.803170,-2.223022e-09,0.155523,4.772745,-1.033628e-10,2016-06-01


### The gamma indicator for the current period is merged with the return for the next period.

In [9]:
testdata=pd.merge(alldata.reset_index(drop=True),allregres,how='inner',on=['Stkcd','Trdmnt'])
testdata

,Stkcd,Msmvosd,Mretwd,Markettype,Ahshrtrd_M,NOCF,ROE,CRGR,Outcap,Trdmnt,const,gamma_NOCF,gamma_ROE,gamma_CRGR,gamma_Outcap
0,2,2.607322e+08,0.026779,4,0.0,5.006252e+10,0.211614,0.181065,6.806587e+09,2020-07-01,0.072719,1.129567e-12,-0.523720,0.138200,-4.313307e-12
1,2,2.649094e+08,0.053335,4,0.0,5.068762e+10,0.209542,0.172512,6.886929e+09,2020-08-01,0.147334,1.255035e-12,-0.943351,0.163595,-2.537399e-12
2,2,2.721951e+08,0.027503,4,0.0,5.131272e+10,0.207470,0.163958,6.967271e+09,2020-09-01,0.157506,1.267777e-12,-0.998311,0.166068,-2.342782e-12
3,2,2.676294e+08,-0.016774,4,0.0,5.193782e+10,0.205398,0.155404,7.047614e+09,2020-10-01,0.154298,1.250741e-12,-0.970607,0.162016,-2.857102e-12
4,2,2.982295e+08,0.114338,4,0.0,5.256292e+10,0.203326,0.146851,7.127956e+09,2020-11-01,0.248219,1.301056e-12,-1.457196,0.171429,-6.480041e-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14132,900952,5.235964e+04,-0.008441,2,0.0,1.277808e+09,0.022635,-0.245765,4.318286e+08,2021-08-01,0.089281,-8.654802e-11,-0.308273,0.006996,-2.776055e-11
14133,900952,5.436490e+04,0.038301,2,0.0,1.341290e+09,0.021815,-0.102230,4.585748e+08,2021-09-01,0.076700,-7.241190e-11,-0.379286,0.006273,-2.083575e-11
14134,900952,5.325087e+04,-0.020489,2,0.0,1.404771e+09,0.020995,0.041305,4.853211e+08,2021-10-01,0.073050,-6.388009e-11,-0.683603,0.004508,-1.313851e-11
14135,900952,5.325087e+04,-0.000002,2,0.0,1.468253e+09,0.020174,0.184840,5.120673e+08,2021-11-01,0.066465,-5.993824e-11,-0.508136,0.005742,-1.374226e-11


<h2> （In-Sample）Quartile composition method

In [10]:
def getgroup(x):
    return pd.qcut(x,10,labels=False)

In [11]:
#decile portfolios
for i in varx:
    testdata['group_'+i] = testdata.groupby('Trdmnt')['gamma_'+i].apply(getgroup)
testdata

,Stkcd,Msmvosd,Mretwd,Markettype,Ahshrtrd_M,NOCF,ROE,CRGR,Outcap,Trdmnt,const,gamma_NOCF,gamma_ROE,gamma_CRGR,gamma_Outcap,group_NOCF,group_ROE,group_CRGR,group_Outcap
0,2,2.607322e+08,0.026779,4,0.0,5.006252e+10,0.211614,0.181065,6.806587e+09,2020-07-01,0.072719,1.129567e-12,-0.523720,0.138200,-4.313307e-12,4,1,8,5
1,2,2.649094e+08,0.053335,4,0.0,5.068762e+10,0.209542,0.172512,6.886929e+09,2020-08-01,0.147334,1.255035e-12,-0.943351,0.163595,-2.537399e-12,4,0,9,5
2,2,2.721951e+08,0.027503,4,0.0,5.131272e+10,0.207470,0.163958,6.967271e+09,2020-09-01,0.157506,1.267777e-12,-0.998311,0.166068,-2.342782e-12,4,0,8,5
3,2,2.676294e+08,-0.016774,4,0.0,5.193782e+10,0.205398,0.155404,7.047614e+09,2020-10-01,0.154298,1.250741e-12,-0.970607,0.162016,-2.857102e-12,4,0,8,5
4,2,2.982295e+08,0.114338,4,0.0,5.256292e+10,0.203326,0.146851,7.127956e+09,2020-11-01,0.248219,1.301056e-12,-1.457196,0.171429,-6.480041e-13,4,0,8,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14132,900952,5.235964e+04,-0.008441,2,0.0,1.277808e+09,0.022635,-0.245765,4.318286e+08,2021-08-01,0.089281,-8.654802e-11,-0.308273,0.006996,-2.776055e-11,1,2,5,3
14133,900952,5.436490e+04,0.038301,2,0.0,1.341290e+09,0.021815,-0.102230,4.585748e+08,2021-09-01,0.076700,-7.241190e-11,-0.379286,0.006273,-2.083575e-11,1,2,5,4
14134,900952,5.325087e+04,-0.020489,2,0.0,1.404771e+09,0.020995,0.041305,4.853211e+08,2021-10-01,0.073050,-6.388009e-11,-0.683603,0.004508,-1.313851e-11,1,1,5,4
14135,900952,5.325087e+04,-0.000002,2,0.0,1.468253e+09,0.020174,0.184840,5.120673e+08,2021-11-01,0.066465,-5.993824e-11,-0.508136,0.005742,-1.374226e-11,1,1,5,4


<h3>Testing the significance of (in-sample) High-minus-low long-short portfolio return alpha

In [12]:
HighLow = pd.DataFrame()
for i in varx:
    gret = testdata.groupby(['Trdmnt','group_'+i])['Mretwd'].mean()
    gretnew = gret.reset_index().pivot_table(index='Trdmnt',columns='group_'+i,values='Mretwd')
    HighLow['spread_'+i] = gretnew.loc[:,9]-gretnew.loc[:,0]
HighLow

,spread_NOCF,spread_ROE,spread_CRGR,spread_Outcap
Trdmnt,,,,
2016-01-01,-0.017159,-0.000702,-0.000077,0.008534
2016-02-01,0.002058,-0.027869,0.008034,-0.007496
2016-03-01,0.017407,-0.022106,-0.035201,0.062649
2016-04-01,-0.000404,0.024422,0.008078,-0.025981
2016-05-01,0.024990,-0.007406,0.000076,0.032054
2016-06-01,-0.017495,-0.019092,0.001024,0.000221
2020-07-01,0.016995,0.005710,0.044034,0.036654
2020-08-01,-0.026424,-0.038432,0.015875,-0.009983
2020-09-01,0.001924,-0.007425,0.026694,0.000953


In [13]:
from scipy import stats
for i in varx:
    print('Result of '+i+': ', stats.ttest_1samp(HighLow['spread_'+i], 0))

Result of NOCF:  Ttest_1sampResult(statistic=0.10740611539333045, pvalue=0.9153982242229375)
Result of ROE:  Ttest_1sampResult(statistic=-1.9768813848112472, pvalue=0.0601640071353427)
Result of CRGR:  Ttest_1sampResult(statistic=1.6892258654248145, pvalue=0.1046888894195058)
Result of Outcap:  Ttest_1sampResult(statistic=4.129434629606404, pvalue=0.0004077446598885244)


In [14]:
for i in varx:
    tmpdata=sm.add_constant(HighLow['spread_'+i])
    mod = sm.OLS(tmpdata.iloc[:,1], tmpdata.iloc[:,0])
    reshac=mod.fit(cov_type='hac',cov_kwds={'maxlags':2}) #Newey-west estimator
    print(reshac.summary())

                            OLS Regression Results                            
Dep. Variable:            spread_NOCF   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Fri, 11 Nov 2022   Prob (F-statistic):                nan
Time:                        13:57:59   Log-Likelihood:                 55.692
No. Observations:                  24   AIC:                            -109.4
Df Residuals:                      23   BIC:                            -108.2
Df Model:                           0                                         
Covariance Type:                  hac                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0005      0.004      0.121      0.9

#### From the above results, only the Alpha of the two factors Return on Equity (ROE) and Capital Expenditure (Output) are statistically significant in the in-sample modelling, suggesting that the returns on the long-short portfolios built through these two factors are significantly non-zero (the regression p-value is significant at the 5 per cent confidence level), which explains to a certain extent the existence of the difference in returns between the stocks, whereas Net Cash Flow from Operations (NOCF) and the Growth Rate of Revenues (CRGR) are not able to explain effectively the differences in the returns between the stocks.

## (Out-of-sample) Rolling estimate of gamma (Return for next period) using 10 years of historical data (current period)

#### Begin rolling regression

In [15]:
import statsmodels.api as sm

allregres2 = pd.DataFrame()

for i in range(test_len):
    train_period = all_period[i:train_len+i]
    traindata = alldata[(alldata.index>=train_period[0]) & (alldata.index<train_period[-1])]

    ##Some filters can be added here
    traindata['nobs'] = traindata.groupby('Stkcd')['Trdmnt'].transform(len)
    traindata=traindata[traindata['nobs']>=30]

    regres2 = traindata.groupby('Stkcd').apply(doreg)
    for j in varx:
        regres2.rename({j:'gamma_'+j},inplace=True,axis=1)
    regres2['Trdmnt'] = str(train_period[-1])[:10]

    allregres2 = pd.concat((allregres2,regres2), axis=0)
    allregres2.dropna(axis=0, inplace=True)

<ipython-input-15-376813d95284>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traindata['nobs'] = traindata.groupby('Stkcd')['Trdmnt'].transform(len)
<ipython-input-15-376813d95284>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traindata['nobs'] = traindata.groupby('Stkcd')['Trdmnt'].transform(len)
<ipython-input-15-376813d95284>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

<ipython-input-15-376813d95284>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traindata['nobs'] = traindata.groupby('Stkcd')['Trdmnt'].transform(len)
<ipython-input-15-376813d95284>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traindata['nobs'] = traindata.groupby('Stkcd')['Trdmnt'].transform(len)
<ipython-input-15-376813d95284>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

### The gamma indicator for the current period is merged with the return for the next period.

In [16]:
testdata2=pd.merge(alldata.reset_index(drop=True),allregres2.reset_index(),on=['Trdmnt','Stkcd'],how='inner')
testdata2

,Stkcd,Msmvosd,Mretwd,Markettype,Ahshrtrd_M,NOCF,ROE,CRGR,Outcap,Trdmnt,const,gamma_NOCF,gamma_ROE,gamma_CRGR,gamma_Outcap
0,2,2.607322e+08,0.026779,4,0.0,5.006252e+10,0.211614,0.181065,6.806587e+09,2020-07-01,0.069817,1.123404e-12,-0.505189,0.136618,-4.522809e-12
1,2,2.649094e+08,0.053335,4,0.0,5.068762e+10,0.209542,0.172512,6.886929e+09,2020-08-01,0.139904,1.239656e-12,-0.896760,0.159859,-3.018989e-12
2,2,2.721951e+08,0.027503,4,0.0,5.131272e+10,0.207470,0.163958,6.967271e+09,2020-09-01,0.157187,1.267119e-12,-0.996341,0.165920,-2.361423e-12
3,2,2.676294e+08,-0.016774,4,0.0,5.193782e+10,0.205398,0.155404,7.047614e+09,2020-10-01,0.166533,1.274366e-12,-1.044997,0.167266,-2.194259e-12
4,2,2.982295e+08,0.114338,4,0.0,5.256292e+10,0.203326,0.146851,7.127956e+09,2020-11-01,0.225615,1.259304e-12,-1.321641,0.162498,-1.783902e-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14122,900952,5.235964e+04,-0.008441,2,0.0,1.277808e+09,0.022635,-0.245765,4.318286e+08,2021-08-01,0.094113,-9.158475e-11,-0.308707,0.007306,-2.985702e-11
14123,900952,5.436490e+04,0.038301,2,0.0,1.341290e+09,0.021815,-0.102230,4.585748e+08,2021-09-01,0.088648,-8.491238e-11,-0.374403,0.006579,-2.613658e-11
14124,900952,5.325087e+04,-0.020489,2,0.0,1.404771e+09,0.020995,0.041305,4.853211e+08,2021-10-01,0.075743,-6.670915e-11,-0.681261,0.004492,-1.435777e-11
14125,900952,5.325087e+04,-0.000002,2,0.0,1.468253e+09,0.020174,0.184840,5.120673e+08,2021-11-01,0.072593,-6.640313e-11,-0.500117,0.005534,-1.656927e-11


## （Out-Of-Sample）Quartile composition method

In [17]:
#decile portfolios
for i in varx:
    testdata2['group_'+i] = testdata2.groupby('Trdmnt')['gamma_'+i].apply(getgroup)
testdata2

,Stkcd,Msmvosd,Mretwd,Markettype,Ahshrtrd_M,NOCF,ROE,CRGR,Outcap,Trdmnt,const,gamma_NOCF,gamma_ROE,gamma_CRGR,gamma_Outcap,group_NOCF,group_ROE,group_CRGR,group_Outcap
0,2,2.607322e+08,0.026779,4,0.0,5.006252e+10,0.211614,0.181065,6.806587e+09,2020-07-01,0.069817,1.123404e-12,-0.505189,0.136618,-4.522809e-12,5,1,8,5
1,2,2.649094e+08,0.053335,4,0.0,5.068762e+10,0.209542,0.172512,6.886929e+09,2020-08-01,0.139904,1.239656e-12,-0.896760,0.159859,-3.018989e-12,4,0,8,5
2,2,2.721951e+08,0.027503,4,0.0,5.131272e+10,0.207470,0.163958,6.967271e+09,2020-09-01,0.157187,1.267119e-12,-0.996341,0.165920,-2.361423e-12,4,0,8,5
3,2,2.676294e+08,-0.016774,4,0.0,5.193782e+10,0.205398,0.155404,7.047614e+09,2020-10-01,0.166533,1.274366e-12,-1.044997,0.167266,-2.194259e-12,4,0,8,5
4,2,2.982295e+08,0.114338,4,0.0,5.256292e+10,0.203326,0.146851,7.127956e+09,2020-11-01,0.225615,1.259304e-12,-1.321641,0.162498,-1.783902e-12,4,0,8,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14122,900952,5.235964e+04,-0.008441,2,0.0,1.277808e+09,0.022635,-0.245765,4.318286e+08,2021-08-01,0.094113,-9.158475e-11,-0.308707,0.007306,-2.985702e-11,0,2,5,3
14123,900952,5.436490e+04,0.038301,2,0.0,1.341290e+09,0.021815,-0.102230,4.585748e+08,2021-09-01,0.088648,-8.491238e-11,-0.374403,0.006579,-2.613658e-11,1,2,5,3
14124,900952,5.325087e+04,-0.020489,2,0.0,1.404771e+09,0.020995,0.041305,4.853211e+08,2021-10-01,0.075743,-6.670915e-11,-0.681261,0.004492,-1.435777e-11,1,1,5,4
14125,900952,5.325087e+04,-0.000002,2,0.0,1.468253e+09,0.020174,0.184840,5.120673e+08,2021-11-01,0.072593,-6.640313e-11,-0.500117,0.005534,-1.656927e-11,1,1,5,4


<h3>Testing the significance of (out-of-sample) High-minus-low long-short portfolio return alpha

In [18]:
HighLow2 = pd.DataFrame()
for i in varx:
    gret2 = testdata2.groupby(['Trdmnt','group_'+i])['Mretwd'].mean()
    gretnew2 = gret2.reset_index().pivot_table(index='Trdmnt',columns='group_'+i,values='Mretwd')
    HighLow2['spread_'+i] = gretnew2.loc[:,9]-gretnew2.loc[:,0]
HighLow2

,spread_NOCF,spread_ROE,spread_CRGR,spread_Outcap
Trdmnt,,,,
2016-01-01,-0.015028,0.010630,0.001224,0.003047
2016-02-01,0.007918,-0.014908,-0.000560,-0.016772
2016-03-01,0.033598,0.011884,-0.015632,-0.004564
2016-04-01,-0.002905,0.000395,0.011464,-0.048982
2016-05-01,0.004783,-0.002771,0.005460,0.006142
2016-06-01,-0.007941,-0.027476,-0.018591,-0.019907
2020-07-01,-0.018858,0.024557,0.034202,0.020213
2020-08-01,-0.024878,-0.017587,0.015753,0.004882
2020-09-01,0.007445,-0.005117,0.024601,-0.015525


In [19]:
from scipy import stats
for i in varx:
    print('Result of '+i+': ', stats.ttest_1samp(HighLow2['spread_'+i], 0))

Result of NOCF:  Ttest_1sampResult(statistic=0.469051330820054, pvalue=0.6434486923383604)
Result of ROE:  Ttest_1sampResult(statistic=0.22263162012801582, pvalue=0.8257874352612)
Result of CRGR:  Ttest_1sampResult(statistic=0.2564007153374597, pvalue=0.799920942330192)
Result of Outcap:  Ttest_1sampResult(statistic=-0.8786951823545806, pvalue=0.3886544051680879)


In [20]:
for i in varx:
    tmpdata2=sm.add_constant(HighLow2['spread_'+i])
    mod2 = sm.OLS(tmpdata2.iloc[:,1], tmpdata2.iloc[:,0])
    reshac2=mod2.fit(cov_type='hac',cov_kwds={'maxlags':2}) #Newey-west estimator
    print(reshac2.summary())

                            OLS Regression Results                            
Dep. Variable:            spread_NOCF   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Fri, 11 Nov 2022   Prob (F-statistic):                nan
Time:                        14:01:50   Log-Likelihood:                 57.130
No. Observations:                  24   AIC:                            -112.3
Df Residuals:                      23   BIC:                            -111.1
Df Model:                           0                                         
Covariance Type:                  hac                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0022      0.004      0.518      0.6

#### In terms of results, out-of-sample modelling indicates that none of the four selected financial indicator factors are able to explain the existence of time-series differential returns between stocks, and that other more effective factors need to be found to explain the phenomenon.

## Spanning Test, calculating factor alpha

#### Assume that the factors modelled within the sample are valid, thus attempting a Spanning Test

#### Merging long-short portfolio return data with factor data

In [21]:
# The reason for constructing a date in the long/short portfolio return data is to change the date to a year-month format to ease the merging process
HighLow['Trdmnt'] = HighLow.index
HighLow['Trdmnt'] = [i[:7] for i in HighLow['Trdmnt']]

# Here the financial indicator factors for each stock are averaged over years and months (probably not very rigorous)
finidx_new = finidx.groupby('Trdmnt').mean()

newdata = pd.merge(HighLow.reset_index(drop=True), finidx_new, on = 'Trdmnt', how = 'inner')
newdata

,spread_NOCF,spread_ROE,spread_CRGR,spread_Outcap,Trdmnt,Stkcd,A100000,A200000,A210000,A210101,...,T40803,T60200,T40902,Surplus,Flua10,Intrsta10,Perdebt,Begdebt,Enddebt,Outcap
0,-0.017159,-0.000702,-0.000077,0.008534,2016-01,306588.305970,3.023418e+10,1.910858e+10,1.279986e+10,2.433004e+09,...,0.079715,0.342980,3.291502,1.366971e+09,0.531978,7.992123e+07,8.555692e+09,8.220937e+09,8.890448e+09,1.510539e+09
1,0.002058,-0.027869,0.008034,-0.007496,2016-02,306588.305970,3.068482e+10,1.941515e+10,1.298942e+10,2.453579e+09,...,0.079912,0.346616,3.123759,1.386229e+09,0.531816,4.985379e+07,8.659886e+09,8.301286e+09,9.018486e+09,1.517195e+09
2,0.017407,-0.022106,-0.035201,0.062649,2016-03,306588.305970,3.113546e+10,1.972172e+10,1.317897e+10,2.474153e+09,...,0.080110,0.350253,2.956017,1.405487e+09,0.531655,1.978635e+07,8.764079e+09,8.381635e+09,9.146524e+09,1.523851e+09
3,-0.000404,0.024422,0.008078,-0.025981,2016-04,306588.305970,3.158611e+10,2.002829e+10,1.336852e+10,2.494728e+09,...,0.080307,0.353889,2.788274,1.424744e+09,0.531494,-1.028109e+07,8.868273e+09,8.461984e+09,9.274562e+09,1.530508e+09
4,0.024990,-0.007406,0.000076,0.032054,2016-05,306588.305970,3.203675e+10,2.033486e+10,1.355808e+10,2.515302e+09,...,0.080504,0.357526,2.620532,1.444002e+09,0.531332,-4.034852e+07,8.972466e+09,8.542333e+09,9.402600e+09,1.537164e+09
5,-0.017495,-0.019092,0.001024,0.000221,2016-06,306588.305970,3.248739e+10,2.064143e+10,1.374763e+10,2.535876e+09,...,0.080702,0.361163,2.452789,1.463259e+09,0.531171,-7.041596e+07,9.076660e+09,8.622682e+09,9.530638e+09,1.543820e+09
6,0.016995,0.005710,0.044034,0.036654,2020-07,336396.385502,8.355646e+10,5.518410e+10,3.741677e+10,4.759022e+09,...,0.062530,0.550085,0.880653,3.649843e+09,0.494023,-2.824614e+08,2.190629e+10,2.104654e+10,2.276604e+10,3.772496e+09
7,-0.026424,-0.038432,0.015875,-0.009983,2020-08,336396.385502,8.420416e+10,5.560422e+10,3.771417e+10,4.764952e+09,...,0.060648,0.547189,0.803431,3.633722e+09,0.493780,-2.253284e+08,2.203820e+10,2.117631e+10,2.290009e+10,3.790772e+09
8,0.001924,-0.007425,0.026694,0.000953,2020-09,336396.385502,8.485186e+10,5.602433e+10,3.801156e+10,4.770883e+09,...,0.058766,0.544294,0.726209,3.617602e+09,0.493536,-1.681954e+08,2.217011e+10,2.130608e+10,2.303414e+10,3.809049e+09
9,-0.003255,-0.012635,-0.000084,0.001181,2020-10,336396.385502,8.549956e+10,5.644445e+10,3.830896e+10,4.776814e+09,...,0.056884,0.541398,0.648988,3.601481e+09,0.493293,-1.110624e+08,2.230202e+10,2.143585e+10,2.316819e+10,3.827325e+09


#### Spanning Test for ROE factor

In [22]:
mod = sm.OLS(newdata.loc[:,'spread_ROE'], sm.add_constant(newdata.iloc[:,-(len(finidx.columns)-2):]))
reshac=mod.fit(cov_type='hac',cov_kwds={'maxlags':2}) #Newey-west estimator
print(reshac.summary())

                            OLS Regression Results                            
Dep. Variable:             spread_ROE   R-squared:                       0.094
Model:                            OLS   Adj. R-squared:                 -0.302
Method:                 Least Squares   F-statistic:                 8.376e+07
Date:                Fri, 11 Nov 2022   Prob (F-statistic):           3.07e-60
Time:                        14:01:51   Log-Likelihood:                 60.395
No. Observations:                  24   AIC:                            -104.8
Df Residuals:                      16   BIC:                            -95.37
Df Model:                           7                                         
Covariance Type:                  hac                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.414e-20   1.59e-20     -2.150      0.0

C:\Users\Myth.God\anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 33, but rank is 15
  warnings.warn('covariance of constraints does not have full '


#### From the results, it can be seen that a large proportion of the factors are able to replicate the ROE factor through some linear combination and the factor Alpha can be found to be significant, suggesting that the ROE factor is not effective in explaining the variation in returns that exists between stocks, as it can also be explained by other factors, which is in line with the principle that the various indicators of the financial statements are intertwined.

#### Next, the Spanning Test was also performed on the Outcap factors.

In [23]:
mod = sm.OLS(newdata.loc[:,'spread_Outcap'], sm.add_constant(newdata.iloc[:,-(len(finidx.columns)-2):]))
reshac=mod.fit(cov_type='hac',cov_kwds={'maxlags':2}) #Newey-west estimator
print(reshac.summary())

                            OLS Regression Results                            
Dep. Variable:          spread_Outcap   R-squared:                       0.283
Model:                            OLS   Adj. R-squared:                 -0.031
Method:                 Least Squares   F-statistic:                -9.473e+07
Date:                Fri, 11 Nov 2022   Prob (F-statistic):               1.00
Time:                        14:01:51   Log-Likelihood:                 53.832
No. Observations:                  24   AIC:                            -91.66
Df Residuals:                      16   BIC:                            -82.24
Df Model:                           7                                         
Covariance Type:                  hac                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.055e-19   3.03e-20      3.485      0.0

C:\Users\Myth.God\anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 33, but rank is 13
  warnings.warn('covariance of constraints does not have full '


#### The Outcap (capex) factor turns out to be very similar to ROE, because although it is relatively effective in explaining the difference in returns between (in-sample) stocks, the fact that the factor Alpha is very significant here suggests that it can also be replicated by the other factors by some linear combination of them, and thus Outcap is not a factor that explains the differential returns of stocks very well either.

<h2>Fama-Macbeth Regression test

### Defining the FM regression model

In [25]:
def dofmreg(data):
    mod = sm.OLS(data['Mretwd'], sm.add_constant(data[varx1]))
    res = mod.fit()
    return res.params

### （In-Sample）Run the FM regression

In [26]:
fmregres = testdata.groupby('Trdmnt').apply(dofmreg)

### Statistical test of regression coefficient Gamma

In [27]:
for i in varx:
    print('Result of '+i+': ', stats.ttest_1samp(fmregres['gamma_'+i], 0))

Result of NOCF:  Ttest_1sampResult(statistic=0.7920875298929982, pvalue=0.43640352304712593)
Result of ROE:  Ttest_1sampResult(statistic=-2.8912448525979357, pvalue=0.008237552918554068)
Result of CRGR:  Ttest_1sampResult(statistic=1.995640381497017, pvalue=0.05794888678328757)
Result of Outcap:  Ttest_1sampResult(statistic=2.123098116531545, pvalue=0.04472275598994361)


#### In terms of the results given by the FM regression, which are similar to the in-sample quantile portfolio results, only the ROE and Outcap factors are statistically relatively more effective in explaining the existence of cross-sectional differential returns between stocks, but only in-sample, as it will be seen later that none of the four factors are very effective in explaining this phenomenon out-of-sample.

### （Out-Of-Sample）Run the FM regression

In [28]:
fmregres2 = testdata2.groupby('Trdmnt').apply(dofmreg)

### Statistical test of regression coefficient Gamma

In [29]:
for i in varx:
    print('Result of '+i+': ', stats.ttest_1samp(fmregres2['gamma_'+i], 0))

Result of NOCF:  Ttest_1sampResult(statistic=-0.06675777325358358, pvalue=0.9473514621503836)
Result of ROE:  Ttest_1sampResult(statistic=-0.45458389833809293, pvalue=0.6536699975903866)
Result of CRGR:  Ttest_1sampResult(statistic=1.874977027515275, pvalue=0.0735501990820145)
Result of Outcap:  Ttest_1sampResult(statistic=-0.21783058975562736, pvalue=0.8294821136019404)


#### The above FM regression results point out that all four factors are unable to statistically significantly explain the difference in returns between stocks in the cross-section, as the p-values of the regression results for all the factors are not significant at the 5% confidence level, suggesting that the factors in the previous period are unable to explain the difference in returns between stocks in the next period, and that other, more effective, factors need to be sought out for the test.